In [ ]:
#Feature Engineering

In [ ]:
import pandas as pd
train = pd.read_csv('data/타이타닉/train.csv')

In [ ]:
train.head()

In [ ]:
before_corr = train.corr(numeric_only=True)
before_corr

In [ ]:
train.isnull().sum()

In [ ]:
test = pd.read_csv('data/타이타닉/test.csv')
test.head()

In [ ]:
test.isnull().sum()

In [ ]:
data_list=[train, test]

In [ ]:
data_list[1].head()

In [ ]:
train['Sex'].head()

In [ ]:
#1. 성별을 코드로 변환한다.
for data in data_list:
    data['Sex'] = data['Sex'].astype('category').cat.codes

In [ ]:
train['Sex'].head(), test['Sex'].head()

In [ ]:
gender_mean = train.groupby('Sex')['Age'].mean()
gender_mean

In [ ]:
#2 나이가 널이면 성별 평균나이로 변환
for data in data_list:
    filt = (data['Sex']==0) & (data['Age'].isnull())
    data.loc[filt, 'Age']=gender_mean[0]
    filt = (data['Sex']==1) & (data['Age'].isnull())
    data.loc[filt, 'Age']=gender_mean[1]

In [ ]:
train['AgeRange'] = pd.cut(train['Age'], 5)
train['AgeRange'].value_counts()

In [ ]:
#3. 나이를 5개의 범주로 변환
for data in data_list:
    filt = data['Age']<=16
    data.loc[filt, 'Age2']=0
    filt = (data['Age']>16) & (data['Age']<=32)
    data.loc[filt, 'Age2']=1
    filt = (data['Age']>32) & (data['Age']<=48)
    data.loc[filt, 'Age2']=2
    filt = (data['Age']>48) & (data['Age']<=64)
    data.loc[filt, 'Age2']=3
    filt = data['Age'] >64
    data.loc[filt, 'Age2']=4

In [ ]:
train['Age2'].value_counts()

In [ ]:
if 'AgeRange' in train.columns:
    train.drop(columns='AgeRange', inplace=True)

In [ ]:
train.columns, test.columns

In [ ]:
for data in data_list:
    if 'Age2' in data.columns:
        data['Age'] = data['Age2']
        data.drop(columns='Age2', inplace=True)

In [ ]:
train['Age'].value_counts()

In [ ]:
train['Name'].head()

In [ ]:
#4. 이름에서 호칭을 뽑아서 코드로 변환

In [ ]:
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\\.')
titles = train['Title'].value_counts()
titles.index

In [ ]:
others = [i for i in titles.index if i not in ['Mr', 'Miss', 'Mrs', 'Master']]
print(others)

In [ ]:
train['Title'] = train['Title'].replace(others, 'Other')
train['Title'].value_counts()

In [ ]:
train['Title_name'] = train['Title'].astype('category').cat.codes
train[['Title', 'Title_name']].value_counts()

In [ ]:
for data in data_list:
    data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\\.')
    titles = data['Title'].value_counts()
    others = [i for i in titles.index if i not in ['Mr', 'Miss', 'Mrs', 'Master']]
    data['Title'] = data['Title'].replace(others, 'Other')
    data['Title_name'] = data['Title'].astype('category').cat.codes

In [ ]:
test[['Title', 'Title_name']].value_counts()

In [ ]:
train.columns

In [ ]:
for data in data_list:
    if 'Name' in data.columns:
        data.drop(columns='Name', inplace=True)
    if 'Title' in data.columns:
        data.drop(columns='Title', inplace=True)

In [ ]:
train.head()

In [ ]:
#5. 배우자, 형제자매수, 부모, 자식수 + 1
for data in data_list:
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

In [ ]:
train['FamilySize'].value_counts()

In [ ]:
train.columns

In [ ]:
for data in data_list:
    if 'SibSp' in data.columns:
        data.drop(columns='SibSp', inplace=True)
    if 'Parch' in data.columns:
        data.drop(columns='Parch', inplace=True)

In [ ]:
train.columns

In [ ]:
train.isnull().sum()

In [ ]:
train['Embarked'].value_counts()

In [ ]:
for data in data_list:
    data.fillna({'Embarked':'S'}, inplace=True)

In [ ]:
train['Embarked'].value_counts()

In [ ]:
test.isnull().sum()

In [ ]:
#6. 항구이름을 코드로 변환
for data in data_list:
    for idx in range(len(data)):
        if data.loc[idx, 'Embarked']=='S':
            data.loc[idx, 'Embarked']=0
        elif data.loc[idx, 'Embarked'] =='C':
            data.loc[idx, 'Embarked']=1
        elif data.loc[idx, 'Embarked'] =='Q':
            data.loc[idx, 'Embarked']=2
        #if not isinstance(data.loc[idx, 'Embarked'], int):
            #data.loc[idx, 'Embarked'] = int(data.loc[idx, 'Embarked'])
    data['Embarked']=data['Embarked'].apply(lambda x: int(x) if not isinstance(x, int) else x)

In [ ]:
train['Embarked'].value_counts()

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.isnull().sum()

In [ ]:
train['Cabin'].value_counts()

In [ ]:
#7. Cabin을 앞자리 1자리로 코드로 변환 
for data in data_list:
    #data['Cabin'] = data['Cabin'].fillna('N')
    data.fillna({'Cabin':'N'}, inplace=True)
    data['Cabin'] = data['Cabin'].apply(lambda x:x[0])
    data['Cabin'] = data['Cabin'].astype('category').cat.codes

In [ ]:
train['Cabin'].value_counts()

In [ ]:
train.head()

In [ ]:
#8. Fare(요금)을 4개의 그룹으로 나누어 코드로 변환
train['FareRange'] = pd.cut(train['Fare'], 4)
train['FareRange'].value_counts()

In [ ]:
for data in data_list:
    filt = data['Fare']<=128
    data.loc[filt, 'Fare2']=0
    filt = (data['Fare']>128) & (data['Fare']<=256)
    data.loc[filt, 'Fare2']=1
    filt = (data['Fare']>256) & (data['Fare']<=384)
    data.loc[filt, 'Fare2']=2
    filt = (data['Fare']>384)
    data.loc[filt, 'Fare2']=3

In [ ]:
train['Fare2'].value_counts()

In [ ]:
train.columns

In [ ]:
for data in data_list:
    if 'Fare2' in data.columns:
        data['Fare'] = data['Fare2']
        data.drop(columns='Fare2', inplace=True)
    if 'FareRange' in data.columns:
        data.drop(columns='FareRange', inplace=True)

    data.fillna({'Fare':0}, inplace=True)

In [ ]:
test.isnull().sum()

In [ ]:
train.columns, test.columns

In [ ]:
#9. 필요없는 칼럼 제거
drop_list=['PassengerId', 'Ticket', 'Cabin']
for data in data_list:
    for col in drop_list:
        if col in data.columns:
            data.drop(columns=col, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.to_csv('data/타이타닉/훈련.csv', index=False)
test.to_csv('data/타이타닉/테스트.csv', index=False)

In [ ]:
before_corr

In [ ]:
train.corr(numeric_only=True)

In [ ]:
train.info()

In [ ]:
train = pd.read_csv('data/타이타닉/훈련.csv')
train.head()

In [ ]:
test = pd.read_csv('data/타이타닉/테스트.csv')
test.head()

In [ ]:
train.corr(numeric_only=True)

In [ ]:
train.head(2)

In [ ]:
train.head(1)

In [ ]:
X_train = train.iloc[:, 1:].values
y_train = train.iloc[:, 0].values

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model= KNeighborsClassifier()
model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)

In [ ]:
X_test = test.iloc[:, :].values
y_test_pred = model.predict(X_test)

In [ ]:
test['Prediction'] = y_test_pred
test.head(1)

In [ ]:
df_test = pd.read_csv('data/타이타닉/test.csv')
df_test['Prediction'] = y_test_pred
df_test.head()

In [ ]:
submission = pd.read_csv('data/타이타닉/gender_submission.csv')
submission['Prediction'] = y_test_pred
submission.to_csv('data/타이타닉/gender_submission.csv', index=False)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/타이타닉/test.csv')
df['pred'] = ''
df.columns

In [ ]:
cols = ['Name', 'Sex', 'Age', 'Fare', 'Pclass', 'Embarked', 'pred']
df = df[cols]
df.columns = ['성명', '성별', '나이', '요금', '등급', '항구', '예측결과']
df.head(10)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model= KNeighborsClassifier()

test = pd.read_csv('data/타이타닉/테스트.csv')
train = pd.read_csv('data/타이타닉/훈련.csv')
X = train.iloc[:, 1:].values
y = train.iloc[:, 0].values
model.fit(X, y)

X_pred = test.iloc[:, :].values
y_pred = model.predict(X_pred)

df_test = pd.read_csv('data/타이타닉/test.csv')
df_test['pred'] = y_pred
df_test.head(5)


In [ ]:
test.head(1)

In [ ]:
train.head(1)

In [ ]:
train = pd.read_csv('data/타이타닉/훈련.csv')
train['Age'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 3))
sns.countplot(x='Age', hue='Survived', data=train, palette='blues')
plt.show()

In [ ]:
train.head()

In [ ]:
def rate(col):
    group = train[col].value_counts()
    df_total = group.to_frame('전체')
    group = train[train['Survived']==1][col].value_counts()
    df_survived = group.to_frame('생존')
    join = df_total.join(df_survived)
    join['사망'] = join['전체']-join['생존']
    join['생존율'] = join['생존']/join['전체']
    return join

In [ ]:
df=rate('Age')
df

In [ ]:
df_train = pd.read_csv('data/타이타닉/train.csv')
df_train.head()

In [ ]:
df_train['AgeRange'] = pd.cut(df_train['Age'], 5)
range = df_train['AgeRange'].value_counts()
range = pd.Series(range)
ranges = range.sort_index().index
index = []
for range in ranges:
    range = str(range).replace('(','').replace(']', '').split(',')
    range = f'{float(range[0]):.0f}~{float(range[1]):.0f}'
    index.append(range)
index

In [ ]:
df=rate('Age')
df= df.sort_index()
df.index = index
df

In [ ]:
import numpy as np
index = np.arange(len(df)) * 3
plt.bar(index+0, df['생존'], width=1)
plt.bar(index+1, df['사망'], width=1)
plt.xticks([x+0.5 for x in index], df.index)
plt.show()